In [71]:
import os
import cv2
import numpy as np
import shutil
import random
import argparse
import pandas as pd
from PIL import Image

In [72]:
#初始設置
images_num = 100
SIZE = 416
images_output_path = "./output"
image_source_path = "./images"
labels_txt_name = "./labels.txt"
small_num = 3
medium_num = 6
big_num = 3

In [73]:
# 把輸出的資料夾淨空
if os.path.exists(images_output_path): 
    shutil.rmtree(images_output_path)
os.mkdir(images_output_path)

In [74]:
#確認圖片生成有沒有衝突
def compute_iou(box1, box2):
    """xmin, ymin, xmax, ymax"""

    A1 = (box1[2] - box1[0])*(box1[3] - box1[1])
    A2 = (box2[2] - box2[0])*(box2[3] - box2[1])

    xmin = max(box1[0], box2[0])
    ymin = max(box1[1], box2[1])
    xmax = min(box1[2], box2[2])
    ymax = min(box1[3], box2[3])

    if ymin >= ymax or xmin >= xmax: return 0
    return  ((xmax-xmin) * (ymax - ymin)) / (A1 + A2)

#回傳麻將的ID
def get_mahjong_ID(jpgName):
    getCSV = pd.read_csv('data.csv',index_col="image-name")
    return getCSV.loc[jpgName].label

#做圖片放上去背景
def make_image(data, image_path, ratio=1):
    #抓目前的狀況
    blank = data[0]
    boxes = data[1]
    label = data[2]
    #看是哪個ID
    ID = get_mahjong_ID(image_path.split("/")[-1])
    #抓圖片
    image = cv2.imread(image_path)
    #放大/縮小
    image = cv2.resize(image, (int(24*ratio), int(32*ratio)))
    #拿圖片的hright width
    h, w, c = image.shape
    #如果跟其他圖片有衝突 換個位置
    while True:
        xmin = np.random.randint(0, SIZE-w, 1)[0]
        ymin = np.random.randint(0, SIZE-h, 1)[0]
        xmax = xmin + w
        ymax = ymin + h
        box = [xmin, ymin, xmax, ymax]

        iou = [compute_iou(box, b) for b in boxes]
        if max(iou) == 0.:
            boxes.append(box)
            label.append(ID)
            break
    #把圖片放上去
    for i in range(w):
        for j in range(h):
            x = xmin + i
            y = ymin + j
            blank[y][x] = image[j][i]
    # 如果想要框框可以放這個 cv2.rectangle(blank, (xmin, ymin), (xmax, ymax), [0, 0, 255], 2)
    # 把這個副函式弄好的傳回去
    data[0] = blank
    data[1] = boxes
    data[2] = label
    return data

In [75]:
image_paths  = [ "./images/" + image_name for image_name in os.listdir(image_source_path)]

with open(labels_txt_name, "w") as wf:
    #算現在跑幾張了
    image_count = 0
    while image_count < images_num: #跑上面設置的總數
        image_path = os.path.realpath(os.path.join(images_output_path, "%04d.jpg" %(image_count+1))) #輸出名字
        annotation = image_path 
        blanks = np.ones(shape=[SIZE, SIZE, 3]) * 255 #白色背景大小
        bboxes = [[0,0,1,1]] #XY位置
        labels = [0] #標籤
        data = [blanks, bboxes, labels]
        
        #圖片START
        # small object 生小
        ratios = [0.6, 0.8]
        N = random.randint(0, small_num)
        for _ in range(N):
            ratio = random.choice(ratios)
            idx = random.randint(1, 627)
            data = make_image(data, image_paths[idx], ratio)
            
        # medium object 生中
        ratios = [1., 1.25, 1.5, 1.75]
        N = random.randint(0, big_num)
        for _ in range(N):
            ratio = random.choice(ratios)
            idx = random.randint(1, 627)
            data = make_image(data, image_paths[idx], ratio)
            
        # medium object 生大
        ratios = [2. , 2.5, 3.]
        N = random.randint(0, medium_num)
        for _ in range(N):
            ratio = random.choice(ratios)
            idx = random.randint(1, 627)
            data = make_image(data, image_paths[idx], ratio)
        #圖片 END
        #生output圖
        cv2.imwrite(image_path, data[0])
        #生txt 檔案
        for i in range(len(labels)):
            if i == 0: continue #第0張是背景
            xmin = str(bboxes[i][0])
            ymin = str(bboxes[i][1])
            xmax = str(bboxes[i][2])
            ymax = str(bboxes[i][3])
            class_ind = str(labels[i])
            annotation += ' ' + ','.join([xmin, ymin, xmax, ymax, str(class_ind)])
            
        
        image_count += 1
        print("=> %s" %annotation)
        wf.write(annotation + "\n")

=> C:\Users\91006\Desktop\yymnist\output\0001.jpg 358,219,377,244,23 272,222,286,241,20 219,359,243,391,42 15,316,39,348,7 153,255,213,335,7 283,75,343,155,11 44,307,92,371,26 219,219,267,283,37 13,176,61,240,41
=> C:\Users\91006\Desktop\yymnist\output\0002.jpg 11,10,53,66,2 187,89,211,121,10 240,197,300,277,1 338,292,410,388,20 98,229,158,309,31 318,15,378,95,30 123,138,183,218,31
=> C:\Users\91006\Desktop\yymnist\output\0003.jpg 28,122,47,147,18 50,265,92,321,21 371,95,407,143,image-name
582.jpg    10
582.jpg    10
Name: label, dtype: int64 64,84,88,116,32 220,80,280,160,8 182,274,242,354,9 284,219,344,299,12 43,154,115,250,22 297,120,369,216,18 35,19,83,83,9
=> C:\Users\91006\Desktop\yymnist\output\0004.jpg 124,177,138,196,42 384,299,408,331,19 72,82,120,146,5 261,64,321,144,9 148,92,196,156,13
=> C:\Users\91006\Desktop\yymnist\output\0005.jpg 260,261,274,280,6 211,217,241,257,12
=> C:\Users\91006\Desktop\yymnist\output\0006.jpg 33,312,47,331,7 240,204,282,260,26 167,22,197,62,9 131

=> C:\Users\91006\Desktop\yymnist\output\0053.jpg 231,246,245,265,31 199,250,213,269,2 299,54,318,79,7 61,7,109,71,31 334,130,406,226,15 188,322,248,402,20
=> C:\Users\91006\Desktop\yymnist\output\0054.jpg 380,365,399,390,36 247,123,261,142,4 257,378,276,403,26 338,158,362,190,13 43,116,85,172,13 303,357,333,397,29 157,130,205,194,18 107,277,155,341,6
=> C:\Users\91006\Desktop\yymnist\output\0055.jpg 27,265,51,297,1 360,94,384,126,34 329,81,353,113,33 112,14,184,110,31 140,139,188,203,38 333,207,405,303,42 67,313,139,409,12 235,58,283,122,5 210,264,270,344,27
=> C:\Users\91006\Desktop\yymnist\output\0056.jpg 211,162,230,187,8 198,20,228,60,13 155,60,191,108,13 162,143,186,175,35 154,222,226,318,16
=> C:\Users\91006\Desktop\yymnist\output\0057.jpg 243,126,257,145,3 260,111,274,130,35 280,63,299,88,31 171,37,243,133,13 60,113,108,177,14 282,246,330,310,33 6,221,54,285,31 305,62,353,126,24 28,4,88,84,12
=> C:\Users\91006\Desktop\yymnist\output\0058.jpg 333,49,347,68,6 56,69,75,94,15 71,29